In [49]:
import cohere
import os
from dotenv import load_dotenv, dotenv_values

In [50]:
load_dotenv()

True

In [51]:
CO_API_KEY = os.environ.get("CO_API_KEY")
co = cohere.ClientV2(CO_API_KEY)

#response = co.chat(model="command-a-03-2025", messages=[{"role": "user", "content": "tell me about university of toronto"}])

#print(response)

In [52]:
prompt_rubric = '''

Given a prompt, can you please improve it so that it follows the Prompt Rubric:

1. First, break the prompt into sequential steps: Eg. First [do something], then [do something]. Finally, [do something].

2. Then, provide necessary context or background information. Eg. Report card. Which report card? For elementary students in Brazil?

3. Afterwards, assign a role to the AI model. Eg. Imagine you are a teacher and I am a student. Please explain Newton’s Law.

4. After this, specify the tone (Please use constructive tone, related to roles: whose perspective
should the response be written from and who is the audience.). If possible, use action words to be more specific.

5. If there are no examples, add an example when applicable.

6. Explicitly say format of the response expected (bullet points, tables, code, etc).

7. Include this at the end of the prompt: “Please do not assume anything, ask me any
clarification questions if need be.

8. Finally, please do not apply it to this prompt. Memorize the Prompt Rubric for future prompts.
'''

In [53]:
response = co.chat(model="command-a-03-2025", messages=[{"role": "user", "content": prompt_rubric}])

In [54]:
print(response.message.content[0].text)

Understood! I’ll memorize the **Prompt Rubric** for future prompts and apply it as needed. Here’s a summary of the rubric for reference:

1. **Sequential Steps**: Break the task into clear, ordered steps (e.g., First [step], then [step], finally [step]).  
2. **Context/Background**: Provide necessary details (e.g., specific subject, audience, or scenario).  
3. **Assign a Role**: Define the AI’s role and the user’s role in the interaction.  
4. **Specify Tone**: Indicate the tone and perspective (e.g., formal, friendly, expert-to-beginner).  
5. **Add Examples**: Include examples if applicable.  
6. **Response Format**: Explicitly state the expected format (e.g., bullet points, paragraphs, tables).  
7. **Clarification Request**: End with “Please do not assume anything, ask me any clarification questions if need be.”  

I’ll apply this rubric to future prompts as required. Let me know if you’d like me to refine or apply it to a specific task!


In [55]:
#prompt = '''explain how to implement Djikstra’s algorithm in C'''
prompt = '''explain object-oriented programming'''

In [56]:
prompt = prompt + 'Improve this prompt using the Prompt Rubric, with the response being a string of the new prompt only.'

In [57]:
response = co.chat(model="command-a-03-2025", messages=[{"role": "user", "content": prompt}])
print(response.message.content[0].text)

"Explain the core principles of object-oriented programming (OOP), including encapsulation, inheritance, polymorphism, and abstraction, and provide examples of how these concepts are applied in real-world programming scenarios."
